In [1]:
phrases = [
    'Wolves eat rabbits.', 
    'The wolf eats the rabbit.', 
    'Some wolves eat vegetables, but most wolves do not eat vegetables.',
    'Trees provide us with wood.',
    'President Obama made great speeches.',
    'Black men and women fight for their rights.'
]

import pandas as pd

df = pd.DataFrame(phrases)

df['id'] = df.index

df.columns = ['doc', 'id']

df

,doc,id
0,Wolves eat rabbits.,0
1,The wolf eats the rabbit.,1
2,"Some wolves eat vegetables, but most wolves do...",2
3,Trees provide us with wood.,3
4,President Obama made great speeches.,4
5,Black men and women fight for their rights.,5


In [2]:
from relatio import Preprocessor

import string
alphabet_string = string.ascii_lowercase
alphabet_list = list(alphabet_string) + ['rt']

import spacy    
nlp = spacy.load("en_core_web_sm")
stop_words = list(nlp.Defaults.stop_words)

stop_words = list(set(stop_words + alphabet_list))

p = Preprocessor(
    spacy_model = "en_core_web_sm",
    remove_punctuation = True,
    remove_digits = True,
    lowercase = True,
    lemmatize = True,
    remove_chars = ["\"",'-',"^",".","?","!",";","(",")",",",":","\'","+","&","|","/","{","}",
                    "~","_","`","[","]",">","<","=","*","%","$","@","#","’"],
    stop_words = stop_words,
    n_process = -1,
    batch_size = 100
)

doc_index, sentences = p.split_into_sentences(
    df, output_path = None, progress_bar = True
)

/home/germain/Desktop/relatio/.tox/dev/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/germain/Desktop/relatio/.tox/dev/lib/python3.7/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


Splitting into sentences...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 37.23it/s]


In [3]:
from relatio import SRL

SRL = SRL(
    path = "https://storage.googleapis.com/allennlp-public-models/openie-model.2020.03.26.tar.gz",
    batch_size = 10,
    cuda_device = -1
)

srl_res = SRL(sentences, progress_bar=True)

Running SRL...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.18it/s]


In [4]:
from relatio import extract_roles

roles, sentence_index = extract_roles(
    srl_res, 
    used_roles = ["ARG0","B-V","B-ARGM-NEG","B-ARGM-MOD","ARG1","ARG2"],
    progress_bar = True
)

for d in roles[0:5]: print(d)

Extracting semantic roles...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 14177.93it/s]

{'ARG0': 'Wolves', 'ARG1': 'rabbits', 'B-V': 'eat'}
{'ARG0': 'The wolf', 'ARG1': 'the rabbit', 'B-V': 'eats'}
{'ARG0': 'Some wolves', 'ARG1': 'vegetables', 'B-V': 'eat'}
{'B-V': 'do'}
{'ARG0': 'most wolves', 'ARG1': 'vegetables', 'B-V': 'eat', 'B-ARGM-NEG': True}


In [5]:
postproc_roles = p.process_roles(roles, 
                                 dict_of_pos_tags_to_keep = {
                                     "ARG0": ['PRON', 'NOUN', 'PROPN'],
                                     "B-V": ['VERB'],
                                     "ARG1": ['NOUN', 'PROPN', 'PRON']
                                 }, 
                                 max_length = 50,
                                 progress_bar = True,
                                 output_path = 'postproc_roles.json')

from relatio.utils import load_roles
postproc_roles = load_roles('postproc_roles.json')

for d in postproc_roles[0:10]: print(d)

Cleaning phrases for role ARG0...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 40.09it/s]


Cleaning phrases for role B-V...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00, 49.79it/s]


Cleaning phrases for role B-ARGM-MOD...


...

Cleaning phrases for role ARG1...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 37.41it/s]


Cleaning phrases for role ARG2...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 13.26it/s]

{'ARG0': 'wolf', 'B-V': 'eat', 'ARG1': 'rabbit'}
{'ARG0': 'wolf', 'B-V': 'eat', 'ARG1': 'rabbit'}
{'ARG0': 'wolf', 'B-V': 'eat', 'ARG1': 'vegetable'}
{}
{'ARG0': 'wolf', 'B-V': 'eat', 'B-ARGM-NEG': True, 'ARG1': 'vegetable'}
{'ARG0': 'tree', 'B-V': 'provide', 'ARG1': 'wood'}
{'ARG0': 'president obama', 'ARG1': 'speech'}
{'ARG0': 'man woman', 'B-V': 'fight', 'ARG2': 'right'}


In [6]:
known_entities = p.mine_entities(
    sentences, 
    clean_entities = True, 
    progress_bar = True,
    output_path = 'entities.pkl'
)

from relatio.utils import load_entities
known_entities = load_entities('entities.pkl')

for n in known_entities.most_common(1): print(n)
    
top_known_entities = [e[0] for e in list(known_entities.most_common(1)) if e[0] != '']

Mining named entities...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 40.09it/s]

('obama', 1)


In [7]:
from relatio.narrative_models_2 import NarrativeModel

pca_args = {'n_components':2, 'random_state':0}

In [9]:
cluster_args = {'min_samples':1,
                'min_cluster_size':2,  
                'cluster_selection_method' : 'eom',
                'metric' : 'euclidean'}

m2 = NarrativeModel(
    model_type = 'hdbscan',
    roles_considered = ['ARG0', 'B-V', 'B-ARGM-NEG', 'ARG1'],
    roles_with_known_entities = ['ARG0','ARG1'],
    known_entities = top_known_entities,
    assignment_to_known_entities = 'character_matching',
    roles_with_unknown_entities = ['ARG0','ARG1'],
    threshold = 0.3
)    

m2.train(postproc_roles, pca_args = pca_args, cluster_args = cluster_args, progress_bar = True)

Embedding phrases...
Computing phrase embeddings...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 30.45it/s]
/home/germain/Desktop/relatio/.tox/dev/lib/python3.7/site-packages/umap/umap_.py:2345: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  "n_neighbors is larger than the dataset size; truncating to "


Clustering phrases into clusters...
Labeling the clusters by the most frequent phrases...
[0, 1]


/home/germain/Desktop/relatio/relatio/narrative_models_2.py:398: RuntimeWarning: Multiple labels for cluster 0- 2 shown: [('wood', 1), ('tree', 1)]. First one is picked.
  RuntimeWarning,


In [10]:
m2.vocab_unknown_entities

{0: Counter({'wood': 1, 'tree': 1}),
 1: Counter({'speech': 1,
          'wolf': 4,
          'rabbit': 2,
          'vegetable': 2,
          'man woman': 1})}

In [11]:
m2.predict(postproc_roles, progress_bar = True)


Predicting entities for role: ARG0...
Matching known entities (with character matching)...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 28422.20it/s]


Matching unknown entities (with embeddings distance)...
Computing phrase embeddings...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 235.50it/s]


Matching unknown entities (with embeddings distance)...
Computing phrase embeddings...


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 352.24it/s]


Matching unknown entities (with embeddings distance)...
Computing phrase embeddings...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 375.19it/s]


Assigning labels to matches...


[{'ARG0': 'wolf', 'B-V': 'eat'},
 {'ARG0': 'wolf', 'B-V': 'eat'},
 {'ARG0': 'wolf', 'B-V': 'eat'},
 {},
 {'ARG0': 'wolf', 'B-V': 'eat', 'B-ARGM-NEG': True},
 {'ARG0': 'tree', 'B-V': 'provide'},
 {'ARG0': 'president obama'},
 {'ARG0': 'man woman', 'B-V': 'fight', 'ARG2': 'right'}]

In [12]:
m2.labels_unknown_entities

{0: 'wood', 1: 'wolf', -1: ''}

In [13]:
#cluster_args = {'n_clusters':5, 'random_state':0}
param_dist = [2,3,4,5]

m2 = NarrativeModel(
    model_type = 'kmeans',
    roles_considered = ['ARG0', 'B-V', 'B-ARGM-NEG', 'ARG1'],
    roles_with_known_entities = ['ARG0','ARG1'],
    known_entities = top_known_entities,
    assignment_to_known_entities = 'character_matching',
    roles_with_unknown_entities = ['ARG0','ARG1'],
    threshold = 0.3
)    

m2.train(postproc_roles, pca_args = pca_args, param_dist = param_dist, progress_bar = True)

Embedding phrases...
Computing phrase embeddings...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 25.64it/s]
/home/germain/Desktop/relatio/.tox/dev/lib/python3.7/site-packages/umap/umap_.py:2345: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  "n_neighbors is larger than the dataset size; truncating to "


Clustering phrases into clusters...
Labeling the clusters by the most frequent phrases...
[0, 1, 2]


/home/germain/Desktop/relatio/relatio/narrative_models_2.py:398: RuntimeWarning: Multiple labels for cluster 0- 2 shown: [('wood', 1), ('tree', 1)]. First one is picked.
  RuntimeWarning,
/home/germain/Desktop/relatio/relatio/narrative_models_2.py:398: RuntimeWarning: Multiple labels for cluster 1- 2 shown: [('speech', 1), ('man woman', 1)]. First one is picked.
  RuntimeWarning,


In [14]:
m2.vocab_unknown_entities

{0: Counter({'wood': 1, 'tree': 1}),
 1: Counter({'speech': 1, 'man woman': 1}),
 2: Counter({'wolf': 4, 'rabbit': 2, 'vegetable': 2})}

In [15]:
m2.predict(postproc_roles, progress_bar = True)


Predicting entities for role: ARG0...
Matching known entities (with character matching)...


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 78085.45it/s]


Matching unknown entities (with embeddings distance)...
Computing phrase embeddings...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 445.79it/s]


Matching unknown entities (with embeddings distance)...
Computing phrase embeddings...


...

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 422.46it/s]


Matching unknown entities (with embeddings distance)...
Computing phrase embeddings...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 427.94it/s]


Assigning labels to matches...


[{'ARG0': 'wolf', 'B-V': 'eat', 'ARG1': 'wolf'},
 {'ARG0': 'wolf', 'B-V': 'eat', 'ARG1': 'wolf'},
 {'ARG0': 'wolf', 'B-V': 'eat', 'ARG1': 'wolf'},
 {},
 {'ARG0': 'wolf', 'B-V': 'eat', 'B-ARGM-NEG': True, 'ARG1': 'wolf'},
 {'ARG0': 'tree', 'B-V': 'provide', 'ARG1': 'wood'},
 {'ARG0': 'president obama', 'ARG1': 'speech'},
 {'ARG0': 'man woman', 'B-V': 'fight', 'ARG2': 'right', 'ARG1': 'speech'}]